Código base para extração dos Arquivos

In [ ]:
pip install tabula-py pandas

In [1]:
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string, get_column_letter
import csv
import pandas as pd

caminho_arquivo = 'arquivo5.xlsx'

workbook = load_workbook(filename=caminho_arquivo)
sheet = workbook.active

C:\Users\mathe\AppData\Local\Temp\ipykernel_6192\434769802.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
#DADOS DO ALUNO

texto_procurado = "NOME DO (A) ALUNO (A)"


coluna = 'D'
coluna_alvo = sheet[f'D']
nomes_encontrados = []
dados_encontrados = []

encontrado = False
for cell in coluna_alvo:
    if cell.value and texto_procurado in str(cell.value):
        # Verifica se a próxima célula na mesma coluna existe e obtém seu valor
        if cell.row < sheet.max_row:
          nome = sheet[f'{coluna}{cell.row + 1}'].value
          forma_ingresso = sheet[f'{"O"}{cell.row + 1}'].value

          if nome not in nomes_encontrados:
            nomes_encontrados.append(nome)
            dados_encontrados.append([nome, forma_ingresso])
          encontrado = True

if not encontrado:
    print("O texto não foi encontrado na coluna especificada.")

data_alunos = 'alunos.csv'

with open(data_alunos, mode='w', newline='', encoding='utf-8') as arquivo_csv:
    writer = csv.writer(arquivo_csv)
    # Cabeçalho do CSV
    writer.writerow(['nome_aluno', 'forma_ingresso'])

    for dados in dados_encontrados:
        writer.writerow(dados)

df = pd.read_csv(data_alunos)

In [ ]:
#GRADE

import csv
import pandas as pd

texto_procurado = "CÓDIGO"
coluna = 'C'
coluna_alvo = sheet[f'{coluna}']

encontrado = False
codigos_encontrados = []
dados_encontrados = []

for cell in coluna_alvo:
    if cell.value and texto_procurado in str(cell.value):
        linha = cell.row + 2
        curriculo = sheet[f'Z{linha - 10}'].value
        if curriculo == 'SI01':
          curriculo = '1'
        else:
          curriculo = '2'

        while True:
            codigo = sheet[f'{coluna}{linha}'].value
            disciplina = sheet[f'E{linha}'].value
            carga = sheet[f'P{linha}'].value
            credito = sheet[f'S{linha}'].value


            if None not in [codigo, disciplina, carga, credito] and codigo not in codigos_encontrados:
                dados_encontrados.append([codigo, disciplina, carga, credito, curriculo])
                codigos_encontrados.append(codigo)

            linha += 1

            if linha > sheet.max_row or texto_procurado in str(sheet[f'{coluna}{linha}'].value):
                break

        # break

# Escrevendo os dados encontrados em um arquivo CSV
data_grade = 'grade.csv'

with open(data_grade, mode='w', newline='', encoding='utf-8') as arquivo_csv:
    writer = csv.writer(arquivo_csv)
    # Cabeçalho do CSV
    writer.writerow(['cod_disciplina', 'nome_disciplina', 'carga_horaria', 'credito', 'curriculo'])

    for dados in dados_encontrados:
        writer.writerow(dados)

print(f"Os dados foram salvos no arquivo CSV: {data_grade}")

# Lendo o arquivo CSV como um DataFrame do pandas
df = pd.read_csv(data_grade)

In [2]:
#HISTORICO

import csv
import pandas as pd

texto_procurado = "NOME DO (A) ALUNO (A)"
texto_divisao = "Curso"
texto_divisao2 = "CH TOTAL DO PERFIL EM HORAS"
coluna = 'D'
linhas_none = 0
coluna_alvo = sheet[f'{coluna}']
encontrado = False
dados_encontrados = []
df_grade = pd.read_csv('grade.csv')

for cell in coluna_alvo:
    if cell.value and texto_procurado in str(cell.value):
        linha = cell.row + 1
        nome  = sheet[f'{coluna}{linha}'].value
        linha = cell.row + 9

        while True:
            if encontrado:
                while linhas_none <= 20:
                    cell.row = linha
                    cellalvo = sheet[f'{coluna}{linha}'].value
                    if  texto_procurado == cellalvo:
                        linhas_none = 0
                        linha = cell.row + 1
                        nome  = sheet[f'{coluna}{linha}'].value
                        linha = cell.row + 9
                        encontrado = False
                        break
                    elif cellalvo is None:
                        linhas_none += 1
                    linha += 1
            
            ano        = sheet[f'A{linha}'].value
            situacao   = sheet[f'Y{linha}'].value
            
            # valor_celula = sheet[f'E{linha}'].value
            # if valor_celula not in df_grade['nome_disciplina'].values:
            #    situacao = sheet[f'E{linha}'].value
            # else:
            #    disciplina = sheet[f'E{linha}'].value

            disciplina = sheet[f'E{linha}'].value
            semestre = sheet[f'B{linha}'].value
            nota = sheet[f'V{linha}'].value
            
            if disciplina is not None:
                dados_encontrados.append([nome, disciplina, ano, nota, situacao, semestre])          
            linha += 1

            if linha < sheet.max_row:
                if texto_divisao in str(sheet[f'J{linha}'].value) or texto_divisao2 in str(sheet[f'A{linha}'].value):
                    encontrado = True
            else:
                break     
        break


# Escrevendo os dados encontrados em um arquivo CSV
data_grade = 'historico.csv'

with open(data_grade, mode='w', newline='', encoding='utf-8') as arquivo_csv:
    writer = csv.writer(arquivo_csv)
    # Cabeçalho do CSV
    writer.writerow(['nome', 'disciplina', 'ano', 'nota', 'situacao', 'semestre'])

    for dados in dados_encontrados:
        writer.writerow(dados)

print(f"Os dados foram salvos no arquivo CSV: {data_grade}")

# Lendo o arquivo CSV como um DataFrame do pandas
df = pd.read_csv(data_grade)



Os dados foram salvos no arquivo CSV: historico.csv


In [ ]:
#LIMPAR HISTORICO 1

import pandas as pd
import re

# Ler o CSV
df = pd.read_csv('historico.csv')

# Iterar sobre as linhas e verificar se o termo "DOCENTE(S)" está presente na coluna 'DISCIPLINA'
for index, row in df.iterrows():
    if 'DOCENTE(S)' in row['disciplina']:
        # Extrair o nome do docente
        match = re.search(r"DOCENTE\(S\): (.+)", row['disciplina'])
        if match:
            nome_docente = match.group(1)
            # Adicionar o nome do docente em uma nova coluna 'Nome_Docente'
            df.at[index, 'nome_Docente'] = nome_docente

# Alternativamente, você pode criar um novo DataFrame apenas com as linhas que contêm "DOCENTE(S)"
linhas_com_docente = df[df['disciplina'].str.contains("DOCENTE(S)")]

# Escrever o DataFrame resultante em um novo arquivo CSV
df.to_csv('historico.csv', index=False)

In [ ]:
# LIMPAR HISTORICO 2

import pandas as pd
import re

# Ler o CSV
df = pd.read_csv('historico.csv')

# Iterar sobre as linhas e verificar se o termo "DOCENTE(S)" está presente na coluna 'disciplina'
for index, row in df.iterrows():
    if 'DOCENTE(S)' in row['disciplina']:
        # Extrair o nome do docente
        match = re.search(r"DOCENTE\(S\): (.+)", row['disciplina'])
        if match:
            nome_docente = match.group(1)
            # Remover o nome do docente e a palavra "DOCENTE(S)" da coluna 'disciplina'
            df.at[index, 'disciplina'] = re.sub(r"DOCENTE\(S\): (.+)", "", row['disciplina']).strip()

# Alternativamente, você pode criar um novo DataFrame apenas com as linhas que contêm "DOCENTE(S)"
linhas_com_docente = df[df['disciplina'].str.contains("DOCENTE(S)")]

# Escrever o DataFrame resultante em um novo arquivo CSV
df.to_csv('historico.csv', index=False)


In [ ]:
# ORDENAR HISTORICO

import pandas as pd

# Ler o CSV
df = pd.read_csv('historico.csv')

# Lista com a ordem desejada das colunas
nova_ordem_colunas = ['nome', 'disciplina', 'nome_Docente', 'ano', 'nota', 'situacao']

# Reordenar as colunas
df = df[nova_ordem_colunas]

# Reordenar os dados com base nas novas colunas
df = df.sort_values(by=nova_ordem_colunas)

# Salvar o DataFrame de volta para um novo arquivo CSV ou substituir o arquivo existente
df.to_csv('historico.csv', index=False)


In [4]:
from openpyxl import load_workbook
from openpyxl.utils import column_index_from_string, get_column_letter
import csv
import pandas as pd

workbook = load_workbook(filename=caminho_arquivo)
sheet = workbook.active

In [5]:
#STATUS ALUNO

import csv
import pandas as pd

texto_procurado = "NOME DO (A) ALUNO (A)"
texto_divisao = "Curso"
texto_divisao2 = "CH TOTAL DO PERFIL EM HORAS"
coluna = 'D'
linhas_none = 0
coluna_alvo = sheet[f'{coluna}']
encontrado = False
dados_encontrados = []
nomes_encontrados = []

for cell in coluna_alvo:
    if cell.value and texto_procurado in str(cell.value):
        linha = cell.row + 1
        nome  = sheet[f'{coluna}{linha}'].value
        linha = cell.row + 9

        while True:
            if encontrado:
                while linhas_none <= 20:
                    cell.row = linha
                    cellalvo = sheet[f'{coluna}{linha}'].value
                    if  texto_procurado == cellalvo:
                        linhas_none = 0
                        linha = cell.row + 1
                        nome  = sheet[f'{coluna}{linha}'].value
                        linha = cell.row + 9
                        encontrado = False
                        break
                    elif cellalvo is None:
                        linhas_none += 1
                    linha += 1
            
            celula_status = sheet[f'E{linha}'].value
            
            if 'DESLIGAMENTO' == celula_status: 
                status = sheet[f'E{linha}'].value
            
            elif 'DESVINCULADO' == celula_status: 
                status = sheet[f'E{linha}'].value
            
            elif 'REINTEGRAÇÃO' == celula_status: 
                status = sheet[f'E{linha}'].value

            elif 'FORMADO' == celula_status: 
                status = sheet[f'E{linha}'].value
       

            linha += 1

            if linha < sheet.max_row:
                if texto_divisao in str(sheet[f'J{linha}'].value) or texto_divisao2 in str(sheet[f'A{linha}'].value):
                    if nome not in nomes_encontrados:
                        nomes_encontrados.append(nome)
                        dados_encontrados.append([nome, status])
                        ##verificar porque  ta saindo nulo
                        status = 'CURSANDO'
                    encontrado = True
            else:
                break

        
        
        break


# Escrevendo os dados encontrados em um arquivo CSV
data_grade = 'status_aluno.csv'

with open(data_grade, mode='w', newline='', encoding='utf-8') as arquivo_csv:
    writer = csv.writer(arquivo_csv)
    # Cabeçalho do CSV
    writer.writerow(['nome', 'status'])

    for dados in dados_encontrados:
        writer.writerow(dados)

print(f"Os dados foram salvos no arquivo CSV: {data_grade}")

# Lendo o arquivo CSV como um DataFrame do pandas
df = pd.read_csv(data_grade)


Os dados foram salvos no arquivo CSV: status_aluno.csv


In [ ]:
#EXTRAINDO PROFESSOR

import pandas as pd

# Leitura do arquivo CSV

df = pd.read_csv('grade.csv')

# Filtrar as linhas que contêm informações sobre docentes
docentes_df = df[df['nome_disciplina'].str.contains("DOCENTE\(S\)")]

# Extrair os nomes dos docentes
nomes_docentes = docentes_df['nome_disciplina'].str.extract(r'DOCENTE\(S\):(.+)$')[0].str.strip()

# Verificar e armazenar os nomes únicos em um conjunto (set)
nomes_unicos = set()
for nome in nomes_docentes:
    if nome not in nomes_unicos:
        nomes_unicos.add(nome)

# Exibir os nomes únicos dos docentes
print("Nomes únicos dos docentes:")
print(nomes_unicos)

# Salvar os nomes únicos dos docentes em um arquivo CSV
nomes_unicos_df = pd.DataFrame(list(nomes_unicos), columns=['Nome'])
nomes_unicos_df.to_csv('professores.csv', index=False)
print("Nomes únicos dos docentes salvos em 'nomes_docentes_unicos.csv'")


In [ ]:
# LIMPAR GRADE

import pandas as pd
import re

# Ler o CSV
df = pd.read_csv('grade.csv')

# Iterar sobre as linhas e verificar se o termo "DOCENTE(S)" está presente na coluna 'nome_disciplina'
for index, row in df.iterrows():
    if 'DOCENTE(S)' in row['nome_disciplina']:
        # Extrair o nome do docente
        match = re.search(r"DOCENTE\(S\): (.+)", row['nome_disciplina'])
        if match:
            nome_docente = match.group(1)
            # Remover o nome do docente e a palavra "DOCENTE(S)" da coluna 'nome_disciplina'
            df.at[index, 'nome_disciplina'] = re.sub(r"DOCENTE\(S\): (.+)", "", row['nome_disciplina']).strip()

# Alternativamente, você pode criar um novo DataFrame apenas com as linhas que contêm "DOCENTE(S)"
linhas_com_docente = df[df['nome_disciplina'].str.contains("DOCENTE(S)")]

# Escrever o DataFrame resultante em um novo arquivo CSV
df.to_csv('grade.csv', index=False)
